# Motor Remaining Useful Life (RUL) Prediction: Baseline Pipeline

This notebook demonstrates a baseline approach for predicting the Remaining Useful Life (RUL) of motors using a dataset similar to NASA C-MAPSS. The workflow includes data loading, feature engineering, model training (XGBoost), and evaluation. 

---

**Outline:**
1. Import libraries and load data
2. Initial data analysis and visualization
3. Add RUL target variable
4. Aggregate features with rolling window
5. Feature engineering: derivatives, rolling stats
6. Train/test split
7. Baseline XGBoost model
8. Model evaluation and error analysis
9. (Optional) Data preparation for LSTM/GRU (sliding window)


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
from IPython.display import display
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_absolute_error
import scipy.stats as st


import shap
pd.set_option('display.max_columns', None)
print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)
np.random.seed(324)




In [ ]:
# Load the dataset
df = pd.read_csv('../data/Data.csv')

# Display the first few rows
df.head()

## 1. Initial Data Analysis and Visualization

- Check data structure, types, and missing values
- Visualize cycle and sensor distributions


In [ ]:
df.columns

In [ ]:
# Show dataframe info and check for missing values
df.info()

In [ ]:
print("\nMissing values per column:")
print(df.isnull().sum())

In [ ]:
# Show basic statistics. Sensors p01 and p00, p07, p09, p10, p16 and p17 are not changed
df.describe().T

In [ ]:
df.nunique()

In [ ]:
print(df.columns[df.nunique() < 2]) # Identify columns with only one unique value

# Drop columns with only one unique value
df = df.loc[:, df.nunique() > 1]

In [ ]:
# Plot distribution of cycles per motor
plt.figure(figsize=(8,4))
df.groupby('id')['cycle'].max().hist(bins=50)
plt.title('Distribution of Maximum Cycles per Motor')
plt.xlabel('Max Cycle')
plt.ylabel('Count')
plt.show()

## 2. Add RUL (Remaining Useful Life) Target Variable

- For each motor, calculate RUL as the difference between the maximum cycle and the current cycle.


In [ ]:
# Calculate RUL for each row 
df_target = df.groupby('id')['cycle'].transform('max')

df['max_cycle'] = df_target.copy()

df['rul'] = df['max_cycle'] - df['cycle']
df.drop('max_cycle', axis=1, inplace=True)

In [ ]:
# ECDF plot for RUL (Remaining Useful Life)
#This plot shows the ECDF (Empirical Cumulative Distribution Function) of RUL. 
# It helps us see the distribution of remaining useful life for all engines.
import seaborn as sns
import matplotlib.pyplot as plt

sns.displot(data=df, x='rul', kind='ecdf')
plt.title('ECDF of RUL')
plt.xlabel('RUL (cycles)')
plt.ylabel('Proportion')
plt.show()

In [ ]:
# Check correlation with RUL. There are p02, p15, p03, p18, p06, p13 that are highly correlated with RUL.
# Cycle type features also strongly correlate with RUL (as expected, since RUL = max_cycle - cycle).
print(df.corr().abs()['rul'].sort_values(ascending=False))
# Plot correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Correlation Matrix')
plt.show()

In [ ]:
df = df.drop(columns=['p20']) # cor with 'p05'

In [ ]:
df.columns

In [ ]:
# Column like as the cycle is time series data, so we need to create rolling features

# Note: When you create rolling window features, the first (window_size-1) rows for each engine will have NaN values.
# This is normal because there is not enough history for the window.
# To use PCA or any model that does not support NaN, you should fill these NaN values.
# The most common way is to fill NaN with the mean or median of the column.

sensor_cols = ['p02', 'p03', 'p04', 'p05', 'p06', 'p08', 'p11', 'p12',
       'p13', 'p14', 'p15', 'p17', 'p18', 'p19', 's1', 's2']
sensor_cols

## 3. Aggregate Features with Rolling Window

- For each motor and each cycle, compute rolling window aggregates (mean, std, min, max) for sensor 
- This helps capture recent trends and variability for each engine.


In [ ]:
# Column like as the cycle is time series data, so we need to create rolling features

# Note: When you create rolling window features, the first (window_size-1) rows for each engine will have NaN values.
# This is normal because there is not enough history for the window.
# To use PCA or any model that does not support NaN, you should fill these NaN values.
# The most common way is to fill NaN with the mean or median of the column.

# Add difference features for each sensor column
for col in sensor_cols:
    df[f'{col}_diff1'] = df.groupby('id')[col].diff()

def add_rolling_features(df, cols, window):
    """ 
    Function to add rolling features for each column in cols
    using a specified window size.
    """
    new_features = {}
    for func in ['mean', 'std', 'min', 'max']:
        for col in cols:
            new_features[f'{col}_roll{window}_{func}'] = (
                df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).agg(func))
            )
    df = pd.concat([df, pd.DataFrame(new_features)], axis=1)
    df = df.copy()  # Defragment the DataFrame
    # Fill NaN values with the mean of the column
    for col in new_features.keys():
        df[col] = df[col].fillna(df[col].mean())
    return df

windows = [5, 10, 20, 30, 45, 60, 90, 120, 180, 350]
# Add rolling features for each window size
for window in windows:
    df = add_rolling_features(df, sensor_cols, window)

df[[c for c in df.columns if 'roll' in c]].head()

In [ ]:
def add_rolling_trend(df, cols, window):
    """
    Function to add rolling trend features for each column in cols
    using a specified window size.
    The trend is calculated using a linear regression fit (slope).
    
    """
    def trend(x):
        idx = np.arange(len(x))
        if len(x) < 2:
            return 0.0
        return np.polyfit(idx, x, 1)[0]
    new_features = {}
    for col in cols:
        new_features[f'{col}_roll{window}_trend'] = (
            df.groupby('id')[col]
              .transform(lambda x: x.rolling(window, min_periods=2).apply(trend, raw=True))
        )
    df = pd.concat([df, pd.DataFrame(new_features, index=df.index)], axis=1)
    df = df.copy()  
    return df


for w in windows:
    df = add_rolling_trend(df, sensor_cols, w)

In [ ]:
from sklearn.impute import SimpleImputer

rolling_cols = [c for c in df.columns if 'roll' in c or 'diff1' in c]
imputer = SimpleImputer(strategy='mean')
df[rolling_cols] = imputer.fit_transform(df[rolling_cols])

## 5. Train/Test Split

- Split the data into training and test sets. We will use the last 20 cycles of each engine as the test set. The rest will be used for training.


In [ ]:
# Create bins for RUL to stratify the split
# This helps to ensure that both training and test sets have a similar distribution of RUL values
# We use pd.qcut to create quantile-based bins, which helps in stratifying the split
# The 'duplicates' parameter is set to 'drop' to avoid issues with bins that have the same edges
# This is useful when the RUL values are not evenly distributed


In [ ]:
# Create bins for RUL to stratify the split
id_rul = df.groupby('id')['rul'].max().reset_index()
id_rul['rul_bin'] = pd.qcut(id_rul['rul'], q=10, duplicates='drop')
id_rul['rul_bin']

In [ ]:
# Split the data into training and test sets, stratifying by the RUL bins
train_ids, test_ids = train_test_split(
    id_rul['id'],
    test_size=0.2,
    random_state=42,
    stratify=id_rul['rul_bin']
)

In [ ]:
# Form train and test sets based on the selected ids
train_df = df[df['id'].isin(train_ids)].copy()
test_df = df[df['id'].isin(test_ids)].copy()

print(train_df['rul'].describe())
print(test_df['rul'].describe())

In [ ]:
# Visualize the distribution of RUL in train and test sets


plt.figure(figsize=(8,4))
plt.hist(train_df['rul'], bins=30, alpha=0.5, label='train')
plt.hist(test_df['rul'], bins=30, alpha=0.5, label='test')
plt.xlabel('RUL')
plt.ylabel('Count')
plt.legend()
plt.title('Train/Test RUL Distribution (no id overlap)')
plt.show()

In [ ]:
from scipy.stats import ks_2samp

stat, p_value = ks_2samp(train_df['rul'], test_df['rul'])
print(f"KS statistic: {stat:.4f}, p-value: {p_value:.4f}")

if p_value > 0.05:
    print("rul distributions in train and test are statistically similar.")
else:
    print("rul distributions in train and test are statistically different.")



In [ ]:
feature_cols = [col for col in train_df.columns if col not in ['rul', 'id', 'cycle']]
X_train = train_df[feature_cols]
y_train = train_df['rul']
X_test = test_df[feature_cols]
y_test = test_df['rul']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# search and remove features with high correlation (> 0.95)
corr_matrix = pd.DataFrame(X_train_scaled, columns=feature_cols).corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print("Will be delete features with high correlation:", to_drop)

# Delete from train/test
X_train_reduced = pd.DataFrame(X_train_scaled, columns=feature_cols).drop(columns=to_drop)
X_test_reduced = pd.DataFrame(X_test_scaled, columns=feature_cols).drop(columns=to_drop)

# 5 LinearRegression

In [ ]:
def print_metrics(y_test, y_pred):
    # Main metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"MAE (all): {mae:.2f}")
    print(f"RMSE (all): {rmse:.2f}")

    # there are some engines with RUL > 200, so we need to analyze errors by ranges
    bins = [0, 60, 100, 200, np.inf]
    labels = ['<=60','60-100','100-200', '>200']
    y_test_bins = pd.cut(y_test, bins=bins, labels=labels)

    for label in labels:
        mask = y_test_bins == label
        if mask.sum() == 0:
            continue
        mae_bin = mean_absolute_error(y_test[mask], np.array(y_pred)[mask])
        rmse_bin = np.sqrt(mean_squared_error(y_test[mask], np.array(y_pred)[mask]))
        print(f"\nRange RUL {label}:")
        print(f"  MAE: {mae_bin:.2f}")
        print(f"  RMSE: {rmse_bin:.2f}")

    # Visualization
    plt.figure(figsize=(8,4))
    plt.scatter(y_test, y_pred, alpha=0.3)
    plt.xlabel('True RUL')
    plt.ylabel('Predicted RUL')
    plt.title('Predicted vs True RUL')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', color='red')
    plt.show()
    # Visualize errors for RUL <= 60
    mask_short = y_test <= 60
    errors = y_test[mask_short] - y_pred[mask_short]

    plt.figure(figsize=(12,6))
    plt.hist(errors, bins=40, alpha=0.7)
    plt.title('(True RUL - Predicted RUL) for RUL ≤ 60')
    plt.xlabel('Error')
    plt.ylabel('Frequency')
    plt.xlim(-60, 30)
    # Calculate errors and visualize confidence intervals 
    
    mean_err = np.mean(errors)
    sem = st.sem(errors)
    plt.axvline(mean_err, color='red', linestyle='--', label='Mean error')
    plt.legend()
    plt.show()
    
    # Weighted MAE
    # We consider errors for RUL <= 60 three times more important
    weights = np.where(y_test <= 60, 3, 1)  
    weighted_mae = np.sum(weights * np.abs(y_test - y_pred)) / np.sum(weights)
    print(f"weighted MAE: {weighted_mae:.2f}")

def plot_feature_importances(model, X_train):
    importances = model.feature_importances_
    feature_names = X_train.columns

    # top 20 most important features
    indices = np.argsort(importances)[::-1][:20]
    plt.figure(figsize=(10,6))
    plt.title("Feature importances (XGBoost)")
    plt.bar(range(len(indices)), importances[indices], align="center")
    plt.xticks(range(len(indices)), [feature_names[i] for i in indices], rotation=90)
    plt.tight_layout()
    plt.show()
    
def plot_feature_importances_catboost(model, X_train):
    importances = model.get_feature_importance()
    feature_names = X_train.columns
    indices = np.argsort(importances)[::-1][:20]
    plt.figure(figsize=(10,6))
    plt.title("Feature importances (CatBoost)")
    plt.bar(range(len(indices)), importances[indices], align="center")
    plt.xticks(range(len(indices)), [feature_names[i] for i in indices], rotation=90)
    plt.tight_layout()
    plt.show()
def plot_shap_values(model, X_test):
    explainer = shap.Explainer(model)
    shap_values = explainer(X_test)

    # Summary plot
    shap.summary_plot(shap_values, X_test, max_display=20)


In [ ]:
def compute_s_score(rul_true, rul_pred):
    """
    Compute S-score based on the difference between predicted and true RUL.
    The S-score is a measure of the accuracy of the RUL predictions.
    It penalizes underestimation more than overestimation.
    """
    diff = rul_pred - rul_true
    return np.sum(np.where(diff < 0, np.exp(-diff/13)-1, np.exp(diff/10)-1))


In [ ]:

lr = LinearRegression()
lr.fit(X_train_reduced, y_train)
y_pred_lr = lr.predict(X_test_reduced)

# S-метрика на тесте
s_score = compute_s_score(y_test.values, y_pred_lr)
print("S-score LinearRegression:", s_score)
print("LinearRegression MAE:", mean_absolute_error(y_test, y_pred_lr))

# Кросс-валидация по S-метрике
s_score_scorer = make_scorer(compute_s_score, greater_is_better=False)
cv_scores_s_lr = cross_val_score(
    lr, X_train_reduced, y_train, cv=5, scoring=s_score_scorer
)
print("Средний S-score LinearRegression по CV:", -cv_scores_s_lr.mean())

In [ ]:
print_metrics(y_test, y_pred_lr)

## 6. Baseline XGBoost Model

- We will train a simple XGBoost model to predict RUL. We use only the rolling window features, derivatives, and PCA features.


In [ ]:
# XGBoost
model_xgb = xgb.XGBRegressor(n_estimators=1000, max_depth=5, learning_rate=0.1, random_state=42, n_jobs=-1)
model_xgb.fit(X_train_reduced, y_train)
y_pred_xgb = model_xgb.predict(X_test_reduced)


s_score_scorer = make_scorer(compute_s_score, greater_is_better=False)
cv_scores_s = cross_val_score(
    model_xgb, X_train_reduced, y_train, cv=5, scoring=s_score_scorer
)
print("Средний S-score по CV:", -cv_scores_s.mean())

# S-метрика на тесте
s_score = compute_s_score(y_test.values, y_pred_xgb)
print("S-score XGBRegressor:", s_score)

In [ ]:
print_metrics(y_test, y_pred_xgb)
plot_feature_importances(model_xgb, X_train_reduced)
plot_shap_values(model_xgb, X_test_reduced)


# 7. Baseline Catboost Model

In [ ]:
# CatBoost

model_cat = CatBoostRegressor(
    early_stopping_rounds=50, iterations=1000, learning_rate=0.1, depth=5, random_seed=42, verbose=100
)

# Cross - validation
s_score_scorer = make_scorer(compute_s_score, greater_is_better=False)

cv_scores_s_cat = cross_val_score(
    model_cat, X_train_reduced, y_train, cv=5, scoring=s_score_scorer
)
print("Средний S-score CatBoost по CV:", -cv_scores_s_cat.mean())

model_cat.fit(X_train_reduced, y_train)
y_pred_cat = model_cat.predict(X_test_reduced)
s_score_cat = compute_s_score(y_test.values, y_pred_cat)
print("S-score CatBoost:", s_score_cat)

In [ ]:
print_metrics(y_test, y_pred_cat)
plot_feature_importances_catboost(model_cat, X_train_reduced)
plot_shap_values(model_cat, X_test_reduced)

Our graph shows the following:  
  
The average error (red dotted line) is negative, i.e. the model overestimates the remaining service life on average (predicts more than it actually is).  
  
Distribution of errors - most errors are in the range from -10 to 10, but there is a long left tail (errors up to -50), i.e. sometimes the model is very wrong in the direction of overestimating the service life.  
  
Practical conclusion:  
The model tends to be "optimistic" - it often believes that the engine will last longer than it actually does. This is dangerous for operation, since you may not have time to replace or service the engine in time.
Recommendation:  
It is worth refining the model or adding a penalty for overestimating the service life to reduce the negative bias of the error.  